# Calculating errors of the Forecasting - Comparing Actual and Predicted Value

**Errors:**

- Error and Mean Error
- Absolute Error and Mean Absolute Error (MAE)
- Squared Error and Mean Squared Error (MSE)
- Squared Error and Root Mean Squared Error (RMSE)
- Percentage Error
- Mean Absolute Percentage Error (MAPE)
- Symmetric Mean Absolute Percentage Error (SMAPE) 

**Measures:**

- Percentage of Correct Direction (PCD)
- Theil's U statistic


<mark>Scale error metrics to a 100% range</mark>

- Mean Error (%)
- Mean Absolute Error (MAE) (%)
- Mean Squared Error (MSE) (%)
- Root Mean Squared Error (RMSE) (%)
- Mean Absolute Percentage Error (MAPE) (%)

### Install packages

In [ ]:
!pip install awswrangler

### Import packages

In [12]:
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

## Load data and pre-processing

In [ ]:
predictions = 'predictions.xlsx'
predictions_df = pd.DataFrame(pd.read_excel(predictions))

print("Shape:", predictions_df.shape)
predictions_df.head(3)

In [ ]:
predictions_subset = predictions_df[['ID', 'Predicted']]

print("Shape:", predictions_subset.shape)
predictions_subset.head(3)

In [ ]:
actual_value = 'actual_values.xlsx'
actual_value_df = pd.DataFrame(pd.read_excel(actual_value))

print("Shape:", actual_value_df.shape)
actual_value_df.head(3)

In [ ]:
actual_value_subset = actual_value_df[['Name', 'id']]

actual_value_subset = actual_value_subset.rename(columns={"id": "ID"})
columns_titles = ['ID', 'Name']
actual_value_subset = actual_value_subset.reindex(columns=columns_titles)
print("Shape:", actual_value_subset.shape)
actual_value_subset.head(3)

In [ ]:
predicted_actual_df = pd.merge(actual_value_subset, predictions_subset, on="ID")

print("Shape:", predicted_actual_df.shape)
predicted_actual_df.head(3)

In [ ]:
wr.s3.to_csv(
    df=predicted_actual_df,
    path='s3:/datasets/time_series/predicted_actual.csv'
)

## Calculating Errors

In [ ]:
predicted_actual_df = wr.s3.read_csv('s3:/datasets/time_series/predicted_actual.csv')

predicted_actual_df = predicted_actual_df.drop(columns=['Unnamed: 0'])
print("Shape:", predicted_actual_df.shape)
predicted_actual_df.head(5)

In [ ]:
# Drop rows with missing values
predicted_actual_df = predicted_actual_df.dropna()

print("Shape:", predicted_actual_df.shape)

In [ ]:
errors_df = predicted_actual_df.copy()

print("Shape:", errors_df.shape)
errors_df.head(3)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Calculate error metrics for each row

# Calculate the error (residual) between actual and predicted values
errors_df['Error'] = errors_df['Actual'] - errors_df['Predicted']
errors_df['Absolute Error'] = np.abs(errors_df['Error'])
errors_df['MAE'] = mean_absolute_error(errors_df['Actual'], errors_df['Predicted'])
errors_df['Squared Error'] = errors_df['Error'] ** 2
errors_df['Percentage Error'] = (errors_df['Error'] / errors_df['Actual']) * 100
#errors_df['Error (%)'] = ((errors_df['Predicted'] - errors_df['Actual']) / errors_df['Actual']) * 100
errors_df['MAPE'] = np.abs(errors_df['Percentage Error'])
# Calculate symmetric mean absolute percentage error (SMAPE) for each row
errors_df['SMAPE'] = (np.abs(errors_df['Actual'] - errors_df['Predicted']) / ((np.abs(errors_df['Actual']) + np.abs(errors_df['Predicted'])) / 2)) * 100

# Calculate Percentage of Correct Direction (PCD) for each row
errors_df['PCD'] = (errors_df['Actual'].diff() > 0) == (errors_df['Predicted'].diff() > 0)
# Convert boolean values to 1 for True and 0 for False
errors_df['PCD'] = errors_df['PCD'].astype(int)

# Calculate Theil's U statistic for each row
errors_df['U'] = np.sqrt(((errors_df['Predicted'] - errors_df['Actual']) ** 2).mean()) / np.sqrt(((errors_df['Actual'].diff()) ** 2).mean())


# Calculate error metrics for the whole dataset
mean_error = errors_df['Error'].mean()
mae = errors_df['Absolute Error'].mean()
mse = errors_df['Squared Error'].mean()
rmse = np.sqrt(mse)
mape = np.mean(np.abs(errors_df['Percentage Error']))
overall_error = ((errors_df['Predicted'] - errors_df['Actual']).sum() / errors_df['Actual'].sum()) * 100
# Calculate overall SMAPE for the whole dataset
overall_smape = (np.abs(errors_df['Actual'] - errors_df['Predicted']).sum() / ((np.abs(errors_df['Actual']) + np.abs(errors_df['Predicted'])).sum() / 2)) * 100
# Calculate PCD for the whole dataset
overall_pcd = errors_df['PCD'].sum() / len(errors_df) * 100
# Calculate Theil's U statistic for the whole dataset
overall_u = np.sqrt(((errors_df['Predicted'] - errors_df['Actual']) ** 2).mean()) / np.sqrt(((errors_df['Actual'].diff()) ** 2).mean())


print('Mean Error:', mean_error)
print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)
print('Mean Absolute Percentage Error (MAPE):', mape)
print('Overall SMAPE:', overall_smape)
print('Overall Error (%):', overall_error)
print('Overall PCD:', overall_pcd)
print('Overall U:', overall_u) # Display Theil's U statistic for the whole dataset

print('\n\n')


# Scale error metrics to a 100% range
mean_error_percent = (mean_error / errors_df['Actual'].mean()) * 100
mae_percent = (mae / errors_df['Actual'].mean()) * 100
mse_percent = (mse / errors_df['Actual'].mean() ** 2) * 100
rmse_percent = (rmse / errors_df['Actual'].mean()) * 100
mape_percent = mape

# Print error metrics for the whole dataset
print('Mean Error (%):', mean_error_percent)
print('Mean Absolute Error (MAE) (%):', mae_percent)
print('Mean Squared Error (MSE) (%):', mse_percent)
print('Root Mean Squared Error (RMSE) (%):', rmse_percent)
print('Mean Absolute Percentage Error (MAPE) (%):', mape_percent)

<mark>Calculate error metrics for the whole dataset</mark>

- **Mean Error:** ``
- **Mean Absolute Error (MAE):** ``
- **Mean Squared Error (MSE):** ``
- **Root Mean Squared Error (RMSE):** ``
- **Mean Absolute Percentage Error (MAPE):** `inf`
- **Overall SMAPE:** ``
- **Overall Error (%):** ``
- **Overall PCD:** ``
- **Overall U:** ``


<mark>Scale error metrics to a 100% range</mark>

- **Mean Error (%):** ``
- **Mean Absolute Error (MAE) (%):** ``
- **Mean Squared Error (MSE) (%):** ``
- **Root Mean Squared Error (RMSE) (%):** ``
- **Mean Absolute Percentage Error (MAPE) (%):** `inf`

In [ ]:
print("Shape:", errors_df.shape)
errors_df.head(5)

---

In [73]:
errors_df.to_excel("errors.xlsx", sheet_name='Errors Metrics')  

In [63]:
wr.s3.to_csv(
    df=errors_df,
    index = False
    path='s3:/datasets/time_series/errors_metrics.csv'
)

{'paths': ['s3://ds-dataset/Penetration_Prediction/Penetration_2022_Q-4/2022-Q4_errors.csv'],
 'partitions_values': {}}

In [ ]:
errors_df = wr.s3.read_csv('s3:/datasets/time_series/errors_metrics.csv')

print("Shape:", errors_df.shape)
errors_df.head(5)

## Calculate Margin of Error

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats


# Calculate the mean and standard deviation of the error
error_mean = errors_df['Error'].mean()
error_std = errors_df['Error'].std()

# Define the desired confidence level (e.g., 95%)
confidence_level = 0.95

# Calculate the critical value based on the confidence level
z_score = stats.norm.ppf((1 + confidence_level) / 2)

# Calculate the margin of error
margin_of_error = z_score * (error_std / np.sqrt(len(df)))

print('Margin of Error:', margin_of_error)

## Charts

### Distribution of Mean Absolute Error (MAE)

In [ ]:
# Removing anything below actual of 1
df = errors_df[errors_df['Actual'] >= 1.0]

# Define the bins
bins = np.arange(0, 101, 10)

# Create a new column 'bins' in the dataframe to hold the bin each 'actual' value falls into
df['bins'] = pd.cut(df['Actual'], bins, right=False)

# Group by bins
grouped = df.groupby('bins')

# Calculate the Mean Absolute Error for each bin
mae_values = grouped.apply(lambda g: mean_absolute_error(g['Actual'], 
                                                         g['Predicted']))

# Plot histogram
plt.bar(range(len(mae_values)), mae_values, color='lightblue')

# Add MAE values on top of each bar
for i, v in enumerate(mae_values):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom')

# Calculate and add total MAE to title
total_mae = mean_absolute_error(df['Actual'], df['Predicted'])
plt.title(f'Total MAE: {total_mae:.2f}')

# Set x-axis labels to bin ranges
plt.xticks(range(len(mae_values)), [str(i) for i in grouped.groups.keys()], rotation=45)

# Set x and y labels
plt.xlabel('Actual Value Bins')
plt.ylabel('Mean Absolute Error')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Count of data points in each bin
counts = df['bins'].value_counts().sort_index()

# Plot histogram
plt.bar(range(len(counts)), counts, color='lightgreen')

# Add count values on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f'{v}', ha='center', va='bottom')

# Set x-axis labels to bin ranges
plt.xticks(range(len(counts)), [str(i) for i in counts.index], rotation=45)

# Set x and y labels
plt.xlabel('Actual Value Bins')
plt.ylabel('Count of Data Points')

# Show the plot
plt.tight_layout()
plt.show()


### Distribution of Mean Squared Error (MSE)

In [ ]:
# Removing anything below actual of 1
df = errors_df[errors_df['Actual'] >= 1.0] # 0

# Define the bins
bins = np.arange(0, 101, 10)

# Create a new column 'bins' in the dataframe to hold the bin each 'actual' value falls into
df['bins'] = pd.cut(df['Actual'], bins, right=False)

# Group by bins
grouped = df.groupby('bins')

# Calculate the Mean Squared Error (MSE) for each bin
mse_values = grouped.apply(lambda g: mean_squared_error(g['Actual'], 
                                                         g['Predicted']))

# Plot histogram
plt.bar(range(len(mse_values)), mse_values, color='lightblue')

# Add MSE values on top of each bar
for i, v in enumerate(mse_values):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom')

# Calculate and add total MSE to title
total_mse = mean_squared_error(df['Actual'], df['Predicted'])
plt.title(f'Total MSE: {total_mse:.2f}')

# Set x-axis labels to bin ranges
plt.xticks(range(len(mse_values)), [str(i) for i in grouped.groups.keys()], rotation=45)

# Set x and y labels
plt.xlabel('Actual Value Bins')
plt.ylabel('Mean Squared Error (MSE)')

# Show the plot
plt.tight_layout()
plt.show()

### Distribution of Root Mean Squared Error (RMSE)

In [ ]:
# Removing anything below actual of 1
df = errors_df[errors_df['Actual'] >= 0] # 0

# Define the bins
bins = np.arange(0, 101, 10)

# Create a new column 'bins' in the dataframe to hold the bin each 'actual' value falls into
df['bins'] = pd.cut(df['Actual'], bins, right=False)

# Group by bins
grouped = df.groupby('bins')

# Calculate the Root Mean Squared Error (RMSE) for each bin
rmse_values = grouped.apply(lambda g: sqrt(mean_squared_error(g['Actual'], 
                                                         g['Predicted'])))

# Plot histogram
plt.bar(range(len(rmse_values)), rmse_values, color='lightblue')

# Add RMSE values on top of each bar
for i, v in enumerate(rmse_values):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom')

# Calculate and add total MSE to title
total_rmse = sqrt(mean_squared_error(df['Actual'], df['Predicted']))
plt.title(f'Total RMSE: {total_rmse:.2f}')

# Set x-axis labels to bin ranges
plt.xticks(range(len(rmse_values)), [str(i) for i in grouped.groups.keys()], rotation=45)

# Set x and y labels
plt.xlabel('Actual Value Bins')
plt.ylabel('Root Mean Squared Error (RMSE)')

# Show the plot
plt.tight_layout()
plt.show()